# User Analytics in the Telecommunication Industry¶ #

## Task 3 - Experience Analytics ##

The Telecommunication industry has experienced a great revolution since the last decade. Mobile devices have become the new fashion trend and play a vital role in everyone's life. The success of the mobile industry is by and large dependent on its consumers. Therefore, it is necessary for the vendors to focus on their target audience i.e. what are the needs and requirements of their consumers and how they feel and perceive their products. Tracking & evaluation of customers’ experience can help the organizations to optimize their products and services so that it meets the evolving user expectations, needs, and acceptance.

In the telecommunication industry, the user experience is related, most of the time, to network parameter performances or the customers’ device characteristics.  
In this section, we will focus on network parameters like TCP retransmission, Round Trip Time (RTT), Throughput, and the customers’ device characteristics like the handset type to conduct a deep user experience analysis.

In [1]:
#importing libraries
!pip install tabulate
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from statistics import *
from tabulate import tabulate
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
#Reading the data
data=pd.read_excel("Week2_challenge_data_source.xlsx")

In [3]:
data.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,13114483460844900352,2019-04-04 12:01:18,770.0,2019-04-25 14:35:31,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9164566995485190,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,13114483482878900224,2019-04-09 13:04:04,235.0,2019-04-25 08:15:48,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,13114483484080500736,2019-04-09 17:42:11,1.0,2019-04-25 11:58:13,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,13114483485442799616,2019-04-10 00:31:25,486.0,2019-04-25 07:36:35,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,13114483499480700928,2019-04-12 20:10:23,565.0,2019-04-25 10:40:32,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [4]:
data1=data[['MSISDN/Number','Avg RTT DL (ms)','Avg RTT UL (ms)','TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)','Handset Type','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)']]

In [5]:
data1.head()

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps)
0,3.366496e+10,42.0,5.0,NaN,NaN,Samsung Galaxy A5 Sm-A520F,23.0,44.0
1,3.368185e+10,65.0,5.0,NaN,NaN,Samsung Galaxy J5 (Sm-J530),16.0,26.0
2,3.376063e+10,NaN,NaN,NaN,NaN,Samsung Galaxy A8 (2018),6.0,9.0
3,3.375034e+10,NaN,NaN,NaN,NaN,undefined,44.0,44.0
4,3.369980e+10,NaN,NaN,NaN,NaN,Samsung Sm-G390F,6.0,9.0


### Detecting Missing values and Outliers And replacing ###

In [6]:
data1.isnull().any(axis=0)

MSISDN/Number                  True
Avg RTT DL (ms)                True
Avg RTT UL (ms)                True
TCP DL Retrans. Vol (Bytes)    True
TCP UL Retrans. Vol (Bytes)    True
Handset Type                   True
Avg Bearer TP DL (kbps)        True
Avg Bearer TP UL (kbps)        True
dtype: bool

In [7]:
#total number of missing values in the columns
data1.isnull().sum(axis=0)

MSISDN/Number                   1066
Avg RTT DL (ms)                27829
Avg RTT UL (ms)                27812
TCP DL Retrans. Vol (Bytes)    88146
TCP UL Retrans. Vol (Bytes)    96649
Handset Type                     572
Avg Bearer TP DL (kbps)            1
Avg Bearer TP UL (kbps)            1
dtype: int64

In [8]:
#total number of missing values in the rows
data1.isnull().sum(axis=1)

0         2
1         2
2         4
3         4
4         4
5         4
6         2
7         1
8         4
9         0
10        4
11        0
12        4
13        4
14        0
15        0
16        2
17        2
18        2
19        4
20        0
21        0
22        2
23        4
24        4
25        2
26        2
27        4
28        2
29        2
         ..
149971    0
149972    1
149973    0
149974    2
149975    0
149976    4
149977    2
149978    2
149979    0
149980    0
149981    4
149982    0
149983    2
149984    2
149985    4
149986    1
149987    4
149988    2
149989    0
149990    0
149991    2
149992    2
149993    2
149994    4
149995    0
149996    2
149997    2
149998    2
149999    2
150000    8
Length: 150001, dtype: int64

In [9]:
# Impute the missing values with Mean
data1['Avg RTT DL (ms)'].fillna(value=data1['Avg RTT DL (ms)'].mean(), inplace=True)
data1['Avg RTT UL (ms)'].fillna(value=data1['Avg RTT UL (ms)'].mean(), inplace=True)
data1['TCP DL Retrans. Vol (Bytes)'].fillna(value=data1['TCP DL Retrans. Vol (Bytes)'].mean(), inplace=True)
data1['TCP UL Retrans. Vol (Bytes)'].fillna(value=data1['TCP UL Retrans. Vol (Bytes)'].mean(), inplace=True)
data1['Avg Bearer TP DL (kbps)'].fillna(value=data1['Avg Bearer TP DL (kbps)'].mean(), inplace=True)
data1['Avg Bearer TP UL (kbps)'].fillna(value=data1['Avg Bearer TP UL (kbps)'].mean(), inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [10]:
# Impute the categorical data with Mode
data1['Handset Type'].fillna(value=data1['Handset Type'].mode().iloc[0], inplace=True)
data1['MSISDN/Number'].fillna(value=data1['MSISDN/Number'].mode().iloc[0], inplace=True)

### Identifying Outliers with Interquartile Range (IQR) 3##

In [11]:
#calculating the interquartile range for each of the variables in the dataset.
Q1 = data1.quantile(0.25)
Q3 = data1.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

MSISDN/Number                  3.213310e+07
Avg RTT DL (ms)                7.479571e+01
Avg RTT UL (ms)                1.466288e+01
TCP DL Retrans. Vol (Bytes)    1.947698e+07
TCP UL Retrans. Vol (Bytes)    6.966497e+05
Avg Bearer TP DL (kbps)        1.966700e+04
Avg Bearer TP UL (kbps)        1.073000e+03
dtype: float64


In [12]:
#Below we generate an output with the 'True' and 'False' values.
#Points where the values are 'True' represent the presence of the outlier.
print(data1 < (Q1 - 1.5 * IQR)) |(data1 > (Q3 + 1.5 * IQR))

        Avg Bearer TP DL (kbps)  Avg Bearer TP UL (kbps)  Avg RTT DL (ms)  \
0                         False                    False            False   
1                         False                    False            False   
2                         False                    False            False   
3                         False                    False            False   
4                         False                    False            False   
5                         False                    False            False   
6                         False                    False            False   
7                         False                    False            False   
8                         False                    False            False   
9                         False                    False            False   
10                        False                    False            False   
11                        False                    False            False   

TypeError: cannot compare a dtyped [bool] array with a scalar of type [NoneType]

### Identifying Outliers using Box plots ###

In [ ]:

df = px.data.tips()
fig = px.box(data1,  y="Avg RTT DL (ms)")
fig.show()

In [ ]:

df = px.data.tips()
fig = px.box(data1, y="Avg RTT DL (ms)")
fig.show()

In [ ]:

df = px.data.tips()
fig = px.box(data1, y="TCP DL Retrans. Vol (Bytes)")
fig.show()

In [ ]:

df = px.data.tips()
fig = px.box(data1, y="TCP UL Retrans. Vol (Bytes)")
fig.show()

### Correcting outliers with the median ###

In [ ]:
#Printing the median and 95 percentile of Avg RTT DL (ms)
print(data1['Avg RTT DL (ms)'].quantile(0.50)) 
print(data1['Avg RTT DL (ms)'].quantile(0.95)) 

In [ ]:
#Replacing outliers with median
data1['Avg RTT DL (ms)'] = np.where(data1['Avg RTT DL (ms)'] > 220, 54, data1['Avg RTT DL (ms)'])

In [ ]:
#Printing the median and 95 percentile of Avg RTT UL (ms)
print(data1['Avg RTT UL (ms)'].quantile(0.50)) 
print(data1['Avg RTT UL (ms)'].quantile(0.95)) 

In [ ]:
#Replacing outliers with the median
data1['Avg RTT UL (ms)'] = np.where(data1['Avg RTT UL (ms)'] > 40, 7, data1['Avg RTT UL (ms)'])

In [ ]:
#Printing the median and 95 percentile of TCP DL Retrans. Vol (Bytes)
print(data1['TCP DL Retrans. Vol (Bytes)'].quantile(0.50)) 
print(data1['TCP DL Retrans. Vol (Bytes)'].quantile(0.95)) 

In [ ]:
#Replacing with median
data1['TCP DL Retrans. Vol (Bytes)'] = np.where(data1['TCP DL Retrans. Vol (Bytes)'] > 20809914.27034193, 20809914.27034193, data1['TCP DL Retrans. Vol (Bytes)'])

In [ ]:
#Printing median and 95 percentile of TCP UL Retrans. Vol (Bytes)
print(data1['TCP UL Retrans. Vol (Bytes)'].quantile(0.50)) 
print(data1['TCP UL Retrans. Vol (Bytes)'].quantile(0.95)) 

In [ ]:
#Replacing with median
data1['TCP UL Retrans. Vol (Bytes)'] = np.where(data1['TCP UL Retrans. Vol (Bytes)'] > 759658.6648110661, 759658.6648110661, data1['TCP UL Retrans. Vol (Bytes)'])

### Aggregating experience metrics per customer ###

In [87]:

data1.to_csv('experience_datamart',index=False)
data1.head()

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Average TCP retransmission,Average RTT,Average throughput
0,3.366496e+10,42.000000,5.000000,2.080991e+07,759658.664811,Samsung Galaxy A5 Sm-A520F,23.0,44.0,2.156957e+07,47.000000,46.0
1,3.368185e+10,65.000000,5.000000,2.080991e+07,759658.664811,Samsung Galaxy J5 (Sm-J530),16.0,26.0,2.156957e+07,70.000000,32.0
2,3.376063e+10,109.795706,17.662883,2.080991e+07,759658.664811,Samsung Galaxy A8 (2018),6.0,9.0,2.156957e+07,127.458589,12.0
3,3.375034e+10,109.795706,17.662883,2.080991e+07,759658.664811,undefined,44.0,44.0,2.156957e+07,127.458589,88.0
4,3.369980e+10,109.795706,17.662883,2.080991e+07,759658.664811,Samsung Sm-G390F,6.0,9.0,2.156957e+07,127.458589,12.0


In [15]:
data1['Average TCP retransmission']=data1['TCP DL Retrans. Vol (Bytes)']+data1['TCP UL Retrans. Vol (Bytes)']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [16]:
data1['Average RTT']=data1['Avg RTT DL (ms)']+data1['Avg RTT UL (ms)']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [18]:
data1['Average throughput']=data1['Avg Bearer TP DL (kbps)']+data1['Avg Bearer TP DL (kbps)']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [20]:
datamart=data1[['MSISDN/Number','Handset Type','Average TCP retransmission','Average RTT','Average throughput']]


In [22]:
datamart.head()

,MSISDN/Number,Handset Type,Average TCP retransmission,Average RTT,Average throughput
0,3.366496e+10,Samsung Galaxy A5 Sm-A520F,2.156957e+07,47.000000,46.0
1,3.368185e+10,Samsung Galaxy J5 (Sm-J530),2.156957e+07,70.000000,32.0
2,3.376063e+10,Samsung Galaxy A8 (2018),2.156957e+07,127.458589,12.0
3,3.375034e+10,undefined,2.156957e+07,127.458589,88.0
4,3.369980e+10,Samsung Sm-G390F,2.156957e+07,127.458589,12.0


1. TCP values in the dataset

In [23]:
#sorting data
datamart.sort_values(by=['Average TCP retransmission'],ascending=False, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [42]:
#getting the top 10 TCP values and saving to csv 
TCP_top10=datamart['Average TCP retransmission'].head(10)
TCP_top10.to_frame()
TCP_top10.to_csv("TCP_top10.csv",index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.



In [43]:
#getting the bottom 10 TCP values and saving to csv file
TCP_bottom10=datamart['Average TCP retransmission'].tail(10)
TCP_bottom10.to_frame()
TCP_bottom10.to_csv("TCP_bottom10.csv",index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.



In [35]:
#getting the most frequent TCP values
most_freq=datamart['Average TCP retransmission'].value_counts().index.tolist()

In [37]:
#changing the list to a datafrmae
df1 = pd.DataFrame(most_freq)

In [41]:
#saving to a csv file
most_freq10=df1.head(10)
most_freq10.to_csv("most_freqTCP.csv",index=False)

2. RTT values in the dataset.

In [44]:
#sorting data
datamart.sort_values(by=['Average RTT'],ascending=False, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [51]:
#getting the top 10 TTT values and saving to csv 
TCP_top10=datamart['Average RTT'].head(10)
TCP_top10.to_frame()
TCP_top10.to_csv("TTT_top10.csv",index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.



In [52]:
#getting the bottom 10 TTT values and saving to csv file
TCP_bottom10=datamart['Average RTT'].tail(10)
TCP_bottom10.to_frame()
TCP_bottom10.to_csv("TTT_bottom10.csv",index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.



In [53]:
#getting the most frequent TTT values
most_freq=datamart['Average RTT'].value_counts().index.tolist()

In [54]:
#changing the list to a datafrmae
df1 = pd.DataFrame(most_freq)

In [55]:
#saving to a csv file
most_freq10=df1.head(10)
most_freq10.to_csv("most_freqTTT.csv",index=False)

3. Throughput values in the dataset.

In [56]:
#sorting data
datamart.sort_values(by=['Average throughput'],ascending=False, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [57]:
#getting the top 10 Throughput values and saving to csv 
TCP_top10=datamart['Average throughput'].head(10)
TCP_top10.to_frame()
TCP_top10.to_csv("Throughput_top10.csv",index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.



In [58]:
#getting the bottom 10 Throughput values and saving to csv file
TCP_bottom10=datamart['Average throughput'].tail(10)
TCP_bottom10.to_frame()
TCP_bottom10.to_csv("Throughput_bottom10.csv",index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.



In [59]:
#getting the most frequent Throughput values
most_freq=datamart['Average throughput'].value_counts().index.tolist()

In [60]:
#changing the list to a datafrmae
df1 = pd.DataFrame(most_freq)

In [61]:
#saving to a csv file
most_freq10=df1.head(10)
most_freq10.to_csv("most_freqThroughput.csv",index=False)

### The distribution of Average Throughput per handset type ###

In [63]:
datamart.head()

,MSISDN/Number,Handset Type,Average TCP retransmission,Average RTT,Average throughput
120876,3.366117e+10,Huawei B528S-23A,1.284627e+06,96.0,756320.0
141250,3.369817e+10,undefined,6.129228e+07,49.0,598514.0
143670,3.367251e+10,Samsung Galaxy S9 Sm-G960F Ds,1.075170e+05,44.0,598004.0
149616,3.365169e+10,Apple iPhone Xs Max (A2101),9.104667e+05,38.0,552080.0
92177,3.366856e+10,Apple iPhone Xr (A2105),2.801102e+07,49.0,538724.0


In [64]:
#We groupby our dataset based on the handset type
datamart.groupby('Handset Type')

In [65]:
#finding the count of the handset types
datamart.groupby('Handset Type').count()

,MSISDN/Number,Average TCP retransmission,Average RTT,Average throughput
Handset Type,,,,
A-Link Telecom I. Cubot A5,1,1,1,1
A-Link Telecom I. Cubot Note Plus,1,1,1,1
A-Link Telecom I. Cubot Note S,1,1,1,1
A-Link Telecom I. Cubot Nova,1,1,1,1
A-Link Telecom I. Cubot Power,1,1,1,1
A-Link Telecom I. Cubot R9,1,1,1,1
A-Link Telecom I. Cubot X18 Plus,2,2,2,2
Acer Liquid M220,1,1,1,1
Acer Liquid Z220,1,1,1,1


In [67]:
#Now we can find out the average throughput for each Handset type
hand_type=datamart.groupby('Handset Type')['Average throughput'].sum()

In [69]:
hand_type.to_frame()

,Average throughput
Handset Type,
A-Link Telecom I. Cubot A5,46368.0
A-Link Telecom I. Cubot Note Plus,13054.0
A-Link Telecom I. Cubot Note S,17030.0
A-Link Telecom I. Cubot Nova,111334.0
A-Link Telecom I. Cubot Power,138252.0
A-Link Telecom I. Cubot R9,5724.0
A-Link Telecom I. Cubot X18 Plus,48432.0
Acer Liquid M220,0.0
Acer Liquid Z220,776.0


### The Average TCP retransmission view per Handset Type ###

In [70]:
#Finding out the average TCP retransmission for each Handset type
TCP_view=datamart.groupby('Handset Type')['Average TCP retransmission'].sum()

In [71]:
TCP_view.to_frame()

,Average TCP retransmission
Handset Type,
A-Link Telecom I. Cubot A5,2.156957e+07
A-Link Telecom I. Cubot Note Plus,6.162130e+05
A-Link Telecom I. Cubot Note S,4.141173e+07
A-Link Telecom I. Cubot Nova,1.371700e+05
A-Link Telecom I. Cubot Power,8.041000e+03
A-Link Telecom I. Cubot R9,3.099900e+04
A-Link Telecom I. Cubot X18 Plus,3.451810e+06
Acer Liquid M220,2.156957e+07
Acer Liquid Z220,1.743870e+05


### K-Means clustering on the Experience metrics ###

In [72]:
datamart.head()

,MSISDN/Number,Handset Type,Average TCP retransmission,Average RTT,Average throughput
120876,3.366117e+10,Huawei B528S-23A,1.284627e+06,96.0,756320.0
141250,3.369817e+10,undefined,6.129228e+07,49.0,598514.0
143670,3.367251e+10,Samsung Galaxy S9 Sm-G960F Ds,1.075170e+05,44.0,598004.0
149616,3.365169e+10,Apple iPhone Xs Max (A2101),9.104667e+05,38.0,552080.0
92177,3.366856e+10,Apple iPhone Xr (A2105),2.801102e+07,49.0,538724.0


In [73]:
datamart.describe()

,MSISDN/Number,Average TCP retransmission,Average RTT,Average throughput
count,1.500010e+05,1.500010e+05,150001.000000,150001.000000
mean,4.182414e+10,2.156957e+07,127.458589,26600.091853
std,2.438731e+12,1.184392e+08,565.144762,47943.597270
min,3.360100e+10,8.600000e+01,0.000000,0.000000
25%,3.365094e+10,1.492449e+06,40.000000,86.000000
50%,3.366361e+10,2.156957e+07,67.000000,126.000000
75%,3.368307e+10,2.156957e+07,127.458589,39420.000000
max,8.823971e+14,4.344116e+09,96924.000000,756320.000000


In [75]:
'''# standardizing the data using sklearn
datamart1=datamart[['Average TCP retransmission','Average RTT','Average throughput']]
scaler = StandardScaler()
datamart1_scaled = scaler.fit_transform(datamart1)'''

In [77]:
'''#statistics of standardized data
pd.DataFrame(datamart1_scaled).describe()'''

,0,1,2
count,1.500010e+05,1.500010e+05,1.500010e+05
mean,-8.793721e-17,3.905450e-15,-7.298391e-14
std,1.000003e+00,1.000003e+00,1.000003e+00
min,-1.821150e-01,-2.255334e-01,-5.548224e-01
25%,-1.695147e-01,-1.547548e-01,-5.530286e-01
50%,-6.290658e-17,-1.069793e-01,-5.521943e-01
75%,-6.290658e-17,0.000000e+00,2.673965e-01
max,3.649603e+01,1.712780e+02,1.522043e+01


In [81]:
# k means using 3 clusters and k-means++ initialization
kmeans = KMeans(n_jobs = -1, n_clusters = 3, init='k-means++')
kmeans.fit(datamart1)
pred = kmeans.predict(datamart1)

In [82]:
data_clustered = pd.DataFrame(datamart1)
data_clustered['cluster'] = pred

In [85]:
data_clustered.head()

,Average TCP retransmission,Average RTT,Average throughput,cluster
120876,1.284627e+06,96.0,756320.0,0
141250,6.129228e+07,49.0,598514.0,0
143670,1.075170e+05,44.0,598004.0,0
149616,9.104667e+05,38.0,552080.0,0
92177,2.801102e+07,49.0,538724.0,0


In [84]:
data_clustered['cluster'].value_counts()

0    149595
2       244
1       162
Name: cluster, dtype: int64

In [90]:
#getting the centroid of the least cluster
kmeans.cluster_centers_[1]

array([3.13199983e+09, 1.49370370e+02, 1.26895321e+05])